### Build model:

### Reducing memory load:
- crop images
- reduce batch size
- reduce number of channels (have 4 available)
- reduce number of base filters
- reduce depth

### working configurations:

#### single channel cropped beyond zeros, batch size 1, depth 4, number of base filters 16:
depth = 4  
filter_num = 16  
model = isensee2017_model(input_shape=(1, 144,144,144), n_base_filters=filter_num, depth=depth, dropout_rate=0.3,
                      n_segmentation_levels=3, n_labels=4, optimizer=Adam, initial_learning_rate=5e-4,
                      loss_function=weighted_dice_coefficient_loss, activation_name="sigmoid")

#### 4-channel cropped beyond zeros, batch size 1, depth 4, number of base filters 16:
depth = 4  
filter_num = 16  
model = isensee2017_model(input_shape=(4, 144,144,144), n_base_filters=filter_num, depth=depth, dropout_rate=0.3,
                      n_segmentation_levels=3, n_labels=4, optimizer=Adam, initial_learning_rate=5e-4,
                      loss_function=weighted_dice_coefficient_loss, activation_name="sigmoid")                      

#### 4-channel cropped full size, batch size 1, depth 4, number of base filters 8:
model = isensee2017_model(input_shape=(4, 160, 192, 160), n_base_filters=8, depth=4, dropout_rate=0.3,
                      n_segmentation_levels=3, n_labels=4, optimizer=Adam, initial_learning_rate=5e-4,
                      loss_function=weighted_dice_coefficient_loss, activation_name="sigmoid")

#### 4-channel cropped full size, batch size 1, depth 5, number of base filters 8:
model = isensee2017_model(input_shape=(4, 160, 192, 160), n_base_filters=8, depth=5, dropout_rate=0.3,
                      n_segmentation_levels=3, n_labels=4, optimizer=Adam, initial_learning_rate=5e-4,
                      loss_function=weighted_dice_coefficient_loss, activation_name="sigmoid")

#### 4-channel cropped full size, batch size 1, depth 5, number of base filters 12:
model = isensee2017_model(input_shape=(4, 160, 192, 160), n_base_filters=16, depth=5, dropout_rate=0.3,
                      n_segmentation_levels=3, n_labels=4, optimizer=Adam, initial_learning_rate=5e-4,
                      loss_function=weighted_dice_coefficient_loss, activation_name="sigmoid")

#### 4-channel cropped full size, batch size 2, depth 5, number of base filters 6:
model = isensee2017_model(input_shape=(4, 160, 192, 160), n_base_filters=6, depth=5, dropout_rate=0.3,
                      n_segmentation_levels=3, n_labels=4, optimizer=Adam, initial_learning_rate=5e-4,
                      loss_function=weighted_dice_coefficient_loss, activation_name="sigmoid")

### out of memory configurations:
#### 4-channel cropped full size, batch size 1, depth 5, number of base filters 16:
model = isensee2017_model(input_shape=(4, 160, 192, 160), n_base_filters=16, depth=5, dropout_rate=0.3,
                      n_segmentation_levels=3, n_labels=4, optimizer=Adam, initial_learning_rate=5e-4,
                      loss_function=weighted_dice_coefficient_loss, activation_name="sigmoid")
                      
#### 4-channel cropped full size, batch size 2, depth 5, number of base filters 12:
model = isensee2017_model(input_shape=(4, 160, 192, 160), n_base_filters=12, depth=5, dropout_rate=0.3,
                      n_segmentation_levels=3, n_labels=4, optimizer=Adam, initial_learning_rate=5e-4,
                      loss_function=weighted_dice_coefficient_loss, activation_name="sigmoid")

### Imports:

In [1]:
from functools import partial

from keras.layers import Input, LeakyReLU, Add, UpSampling3D, Activation, SpatialDropout3D
from keras.engine import Model
from keras.optimizers import Adam
from keras.optimizers import RMSprop

import pandas as pd

import numpy as np
from keras import backend as K
from keras.engine import Input, Model
from keras.layers import Conv3D, MaxPooling3D, UpSampling3D, Activation, BatchNormalization, PReLU, Deconvolution3D, Flatten, Dense

# K.set_image_dim_ordering('th')
K.set_image_dim_ordering('tf')
K.set_image_data_format('channels_first')

try:
    from keras.engine import merge
except ImportError:
    from keras.layers.merge import concatenate

Using TensorFlow backend.


### Reading in survival data.csv:

In [2]:
survival_data = pd.read_csv('survival_data.csv')

In [3]:
ID = 'Brats17_TCIA_469_1'
survival_data[survival_data.Brats17ID==ID].Survival.astype(int).values.item(0)

519

### Calculating metrics:

In [4]:
def dice_coefficient(y_true, y_pred, smooth=1.):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coefficient_loss(y_true, y_pred):
    return -dice_coefficient(y_true, y_pred)


def weighted_dice_coefficient(y_true, y_pred, axis=(-3, -2, -1), smooth=0.00001):
    """
    Weighted dice coefficient. Default axis assumes a "channels first" data structure
    :param smooth:
    :param y_true:
    :param y_pred:
    :param axis:
    :return:
    """
    return K.mean(2. * (K.sum(y_true * y_pred,
                              axis=axis) + smooth/2)/(K.sum(y_true,
                                                            axis=axis) + K.sum(y_pred,
                                                                               axis=axis) + smooth))


def weighted_dice_coefficient_loss(y_true, y_pred):
    return -weighted_dice_coefficient(y_true, y_pred)


def label_wise_dice_coefficient(y_true, y_pred, label_index):
    return dice_coefficient(y_true[:, label_index], y_pred[:, label_index])


def get_label_dice_coefficient_function(label_index):
    f = partial(label_wise_dice_coefficient, label_index=label_index)
    f.__setattr__('__name__', 'label_{0}_dice_coef'.format(label_index))
    return f


dice_coef = dice_coefficient
dice_coef_loss = dice_coefficient_loss

In [35]:
def create_convolution_block(input_layer, n_filters, batch_normalization=False, kernel=(3, 3, 3), activation=None,
                             padding='same', strides=(1, 1, 1), instance_normalization=False):
    """
    :param strides:
    :param input_layer:
    :param n_filters:
    :param batch_normalization:
    :param kernel:
    :param activation: Keras activation layer to use. (default is 'relu')
    :param padding:
    :return:
    """
    layer = Conv3D(n_filters, kernel, padding=padding, strides=strides)(input_layer)
    if batch_normalization:
        layer = BatchNormalization(axis=1)(layer)
    elif instance_normalization:
        try:
            from keras_contrib.layers.normalization import InstanceNormalization
        except ImportError:
            raise ImportError("Install keras_contrib in order to use instance normalization."
                              "\nTry: pip install git+https://www.github.com/farizrahman4u/keras-contrib.git")
        layer = InstanceNormalization(axis=1)(layer)
    if activation is None:
        return Activation('relu')(layer)
    else:
        return activation()(layer)


def compute_level_output_shape(n_filters, depth, pool_size, image_shape):
    """
    Each level has a particular output shape based on the number of filters used in that level and the depth or number 
    of max pooling operations that have been done on the data at that point.
    :param image_shape: shape of the 3d image.
    :param pool_size: the pool_size parameter used in the max pooling operation.
    :param n_filters: Number of filters used by the last node in a given level.
    :param depth: The number of levels down in the U-shaped model a given node is.
    :return: 5D vector of the shape of the output node 
    """
    output_image_shape = np.asarray(np.divide(image_shape, np.power(pool_size, depth)), dtype=np.int32).tolist()
    return tuple([None, n_filters] + output_image_shape)


def get_up_convolution(n_filters, pool_size, kernel_size=(2, 2, 2), strides=(2, 2, 2),
                       deconvolution=False):
    if deconvolution:
        return Deconvolution3D(filters=n_filters, kernel_size=kernel_size,
                               strides=strides)
    else:
        return UpSampling3D(size=pool_size)

def create_localization_module(input_layer, n_filters):
    convolution1 = create_convolution_block(input_layer, n_filters)
    convolution2 = create_convolution_block(convolution1, n_filters, kernel=(1, 1, 1))
    return convolution2


def create_up_sampling_module(input_layer, n_filters, size=(2, 2, 2)):
    up_sample = UpSampling3D(size=size)(input_layer)
    convolution = create_convolution_block(up_sample, n_filters)
    return convolution


def create_context_module(input_layer, n_level_filters, dropout_rate=0.3, data_format="channels_first"):
    convolution1 = create_convolution_block(input_layer=input_layer, n_filters=n_level_filters)
    dropout = SpatialDropout3D(rate=dropout_rate, data_format=data_format)(convolution1)
    convolution2 = create_convolution_block(input_layer=dropout, n_filters=n_level_filters)
    return convolution2


create_convolution_block = partial(create_convolution_block, activation=LeakyReLU, instance_normalization=True)


def isensee2017_model(input_shape=(4, 128, 128, 128), n_base_filters=16, depth=5, dropout_rate=0.3,
                      n_segmentation_levels=3, n_labels=4, optimizer=Adam, initial_learning_rate=5e-4,
                      loss_function={'activation_block': weighted_dice_coefficient_loss, 'survival_block': 'mean_squared_error'}, 
                      activation_name="sigmoid"):
    """
    This function builds a model proposed by Isensee et al. for the BRATS 2017 competition:
    https://www.cbica.upenn.edu/sbia/Spyridon.Bakas/MICCAI_BraTS/MICCAI_BraTS_2017_proceedings_shortPapers.pdf
    This network is highly similar to the model proposed by Kayalibay et al. "CNN-based Segmentation of Medical
    Imaging Data", 2017: https://arxiv.org/pdf/1701.03056.pdf
    :param input_shape:
    :param n_base_filters:
    :param depth:
    :param dropout_rate:
    :param n_segmentation_levels:
    :param n_labels:
    :param optimizer:
    :param initial_learning_rate:
    :param loss_function:
    :param activation_name:
    :return:
    """
    inputs = Input(input_shape)

    current_layer = inputs
    level_output_layers = list()
    level_filters = list()
    for level_number in range(depth):
        n_level_filters = (2**level_number) * n_base_filters
        level_filters.append(n_level_filters)

        if current_layer is inputs:
            in_conv = create_convolution_block(current_layer, n_level_filters)
        else:
            in_conv = create_convolution_block(current_layer, n_level_filters, strides=(2, 2, 2))

        context_output_layer = create_context_module(in_conv, n_level_filters, dropout_rate=dropout_rate)

        summation_layer = Add(name='summation_layer')([in_conv, context_output_layer])
        level_output_layers.append(summation_layer)
        current_layer = summation_layer

    segmentation_layers = list()
    for level_number in range(depth - 2, -1, -1):
        up_sampling = create_up_sampling_module(current_layer, level_filters[level_number])
        concatenation_layer = concatenate([level_output_layers[level_number], up_sampling], axis=1)
        localization_output = create_localization_module(concatenation_layer, level_filters[level_number])
        current_layer = localization_output
        if level_number < n_segmentation_levels:
            segmentation_layers.insert(0, create_convolution_block(current_layer, n_filters=n_labels, kernel=(1, 1, 1)))

    output_layer = None
    for level_number in reversed(range(n_segmentation_levels)):
        segmentation_layer = segmentation_layers[level_number]
        if output_layer is None:
            output_layer = segmentation_layer
        else:
            output_layer = Add()([output_layer, segmentation_layer])

        if level_number > 0:
            output_layer = UpSampling3D(size=(2, 2, 2))(output_layer)

    activation_block = Activation(activation = activation_name, name='activation_block')(output_layer)
#     survival_block = Activation("linear")(summation_layer)
#     activation_block = Dense(1, activation=activation_name, name='activation_block')(output_layer)
    flatten = Flatten(name='flatten')(summation_layer)
    survival_block = Dense(1, activation='linear', name='survival_block')(flatten)

    model = Model(inputs=inputs, outputs=[activation_block,survival_block])
#     model.compile(optimizer=optimizer(lr=initial_learning_rate), loss=loss_function)
    model.compile(optimizer=optimizer(lr=initial_learning_rate), loss=loss_function, 
                  loss_weights={'activation_block': 1., 'survival_block': 0.2},
                 metrics={'activation_block': ['accuracy',weighted_dice_coefficient, dice_coefficient], 'survival_block': ['accuracy']})
#     loss={'activation_block': 'binary_crossentropy', 'survival_block': 'mean_squared_error'}
    # assign weights and loss as dictionaries
    # functional-api-guide
    # loss_weights define the ratio of how much I care about optimizing each one
    return model

In [36]:
# change the number of labels?
# loss_function={'activation_block': weighted_dice_coefficient_loss, 'survival_block': 'mean_squared_error'}

model = isensee2017_model(input_shape=(4, 160, 192, 160), n_base_filters=12, depth=5, dropout_rate=0.3,
                      n_segmentation_levels=3, n_labels=3, optimizer=RMSprop, initial_learning_rate=5e-4,
                      loss_function={'activation_block': weighted_dice_coefficient_loss, 'survival_block': 'mean_squared_error'}, 
                          activation_name="sigmoid")
model.summary(line_length=150) # add the parameter that allows me to show everything instead of cutting it off

______________________________________________________________________________________________________________________________________________________
Layer (type)                                     Output Shape                     Param #           Connected to                                      
input_3 (InputLayer)                             (None, 4, 160, 192, 160)         0                                                                   
______________________________________________________________________________________________________________________________________________________
conv3d_61 (Conv3D)                               (None, 12, 160, 192, 160)        1308              input_3[0][0]                                     
______________________________________________________________________________________________________________________________________________________
instance_normalization_61 (InstanceNormalization (None, 12, 160, 192, 160)        24          

## Make the labels and test train dictionaries:

In [37]:
# from glob import glob
# paths = glob('/Users/etheredgej/Desktop/MICCAI_BraTS17_Data_Training/train/HGG/*/')
# print(paths)

import os
HGG_dir_list = next(os.walk('./HGG/'))[1]
print(len(HGG_dir_list))
LGG_dir_list = next(os.walk('./LGG/'))[1]
print(len(LGG_dir_list))

210
75


In [38]:
# completelist = HGG_dir_list + LGG_dir_list

completelist = list(survival_data.Brats17ID.copy())

# print(completelist[0:4])
np.random.shuffle(completelist) # shuffles in place
# print(completelist[0:4])

partition={}

holdout_percentage=0.15
partition['holdout']=completelist[0:int(len(completelist)*holdout_percentage)]
trainlist=completelist[int(len(completelist)*holdout_percentage):len(completelist)]

train_percentage=0.7
partition['train']=trainlist[0:int(len(trainlist)*train_percentage)]
partition['test']=trainlist[int(len(trainlist)*train_percentage):len(trainlist)]


labels={}
# HGG=0
# LGG=1
for directory in HGG_dir_list:
    labels[directory]=0
for directory in LGG_dir_list:
    labels[directory]=1
    
print(len(partition['holdout']))
print(len(partition['train']))
print(len(partition['test']))

24
97
42


In [39]:
# len(completelist)
# len(survival_data.Brats17ID)
# len(set.intersection(set(completelist),set(survival_data.Brats17ID)))

### crop_img function:

In [40]:
import numpy as np
from nilearn.image.image import check_niimg
from nilearn.image.image import _crop_img_to as crop_img_to


def crop_img(img, rtol=1e-8, copy=True, return_slices=False):
    """Crops img as much as possible
    Will crop img, removing as many zero entries as possible
    without touching non-zero entries. Will leave one voxel of
    zero padding around the obtained non-zero area in order to
    avoid sampling issues later on.
    Parameters
    ----------
    img: Niimg-like object
        See http://nilearn.github.io/manipulating_images/input_output.html
        img to be cropped.
    rtol: float
        relative tolerance (with respect to maximal absolute
        value of the image), under which values are considered
        negligeable and thus croppable.
    copy: boolean
        Specifies whether cropped data is copied or not.
    return_slices: boolean
        If True, the slices that define the cropped image will be returned.
    Returns
    -------
    cropped_img: image
        Cropped version of the input image
    """

    img = check_niimg(img)
    data = img.get_data()
    infinity_norm = max(-data.min(), data.max())
    passes_threshold = np.logical_or(data < -rtol * infinity_norm,
                                     data > rtol * infinity_norm)

    if data.ndim == 4:
        passes_threshold = np.any(passes_threshold, axis=-1)
    coords = np.array(np.where(passes_threshold))
    start = coords.min(axis=1)
    end = coords.max(axis=1) + 1

    # pad with one voxel to avoid resampling problems
    start = np.maximum(start - 1, 0)
    end = np.minimum(end + 1, data.shape[:3])

    slices = [slice(s, e) for s, e in zip(start, end)]

    if return_slices:
        return slices

    return crop_img_to(img, slices, copy=copy)

### Data generator:

In [41]:
# https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly.html

import numpy as np
import keras
import nibabel as nib

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, batch_size=2, dim=(240,240,155), n_channels=4,
                 n_classes=3, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y1, y2 = self.__data_generation(list_IDs_temp)

        return X, [y1, y2]

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, self.n_channels, *self.dim))
        y1 = np.empty((self.batch_size, 3, *self.dim))
        y2 = np.empty(self.batch_size)

        # Generate data
        # Decode and load the data
        for i, ID in enumerate(list_IDs_temp):

            img1 = './data/' + ID + '_flair.nii.gz'
            img2 = './data/' + ID + '_t1.nii.gz'
            img3 = './data/' + ID + '_t1ce.nii.gz'
            img4 = './data/' + ID + '_t2.nii.gz'
            img5 = './data/' + ID + '_seg.nii.gz'

            newimage = nib.concat_images([img1, img2, img3, img4, img5])
            cropped = crop_img(newimage)         
            img_array = np.array(cropped.dataobj)
            z = np.rollaxis(img_array, 3, 0)

            padded_image = np.zeros((5,160,192,160))
            padded_image[:z.shape[0],:z.shape[1],:z.shape[2],:z.shape[3]] = z

            a,b,c,d,seg_mask = np.split(padded_image, 5, axis=0)

            images = np.concatenate([a,b,c,d], axis=0)

            # print("images shape:", images.shape, "images values:", np.unique(images.astype(int)))

            # split the channels:
            # seg_mask_1 = copy.deepcopy(seg_mask.astype(int))
            seg_mask_1 = np.zeros((1,160,192,160))
            seg_mask_1[seg_mask.astype(int) == 1] = 1
            seg_mask_2 = np.zeros((1,160,192,160))
            seg_mask_2[seg_mask.astype(int) == 2] = 1
            seg_mask_3 = np.zeros((1,160,192,160))
            seg_mask_3[seg_mask.astype(int) == 4] = 1
            seg_mask_3ch = np.concatenate([seg_mask_1,seg_mask_2,seg_mask_3], axis=0).astype(int)
            
            # 1) the "enhancing tumor" (ET), 2) the "tumor core" (TC), and 3) the "whole tumor" (WT) 
            # The ET is described by areas that show hyper-intensity in T1Gd when compared to T1, but also when compared to “healthy” white matter in T1Gd. The TC describes the bulk of the tumor, which is what is typically resected. The TC entails the ET, as well as the necrotic (fluid-filled) and the non-enhancing (solid) parts of the tumor. The appearance of the necrotic (NCR) and the non-enhancing (NET) tumor core is typically hypo-intense in T1-Gd when compared to T1. The WT describes the complete extent of the disease, as it entails the TC and the peritumoral edema (ED), which is typically depicted by hyper-intense signal in FLAIR.
            # The labels in the provided data are: 
            # 1 for NCR & NET (necrotic (NCR) and the non-enhancing (NET) tumor core) = TC ("tumor core")
            # 2 for ED ("peritumoral edema")
            # 4 for ET ("enhancing tumor")
            # 0 for everything else

            X[i,] = images
            y1[i,] = seg_mask_3ch
#             y2[i,] = int(survival_data[survival_data.Brats17ID==ID].Survival) # list of the survival day values
            y2[i,] = survival_data[survival_data.Brats17ID==ID].Survival.astype(int).values.item(0)
            
#         return X, keras.utils.to_categorical(y, num_classes=self.n_classes)
        return X, y1, y2

In [42]:
import numpy as np

from keras.models import Sequential
# from my_classes import DataGenerator

In [43]:
# Parameters
params = {'dim': (160,192,160),
          'batch_size': 1,
          'n_classes': 3,
          'n_channels': 4,
          'shuffle': True}

# # Datasets
# partition = # IDs
# labels = # Labels

# Generators
training_generator = DataGenerator(partition['train'], **params)
validation_generator = DataGenerator(partition['test'], **params)

# # Design model
# model = Sequential()
# [...] # Architecture
# model.compile()

# Train model on dataset
# model.fit_generator(generator=training_generator,
#                     validation_data=validation_generator,
#                     use_multiprocessing=True,
#                     workers=4)

cb_1=keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto')
cb_2=keras.callbacks.ModelCheckpoint(filepath="weights.{epoch:02d}-{val_loss:.2f}.hdf5", monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)

# losses = {'activation_block': weighted_dice_coefficient_loss, 'survival_block': 'mean_squared_error'}
# cb_1=keras.callbacks.EarlyStopping(monitor=losses, min_delta=0, patience=2, verbose=0, mode='auto')
# cb_2=keras.callbacks.ModelCheckpoint(filepath="weights.{epoch:02d}-{val_loss:.2f}.hdf5", monitor=losses, verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)

results = model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                   epochs=2, 
                   nb_worker=4,
                   callbacks=[cb_1,cb_2])

# results = model.fit_generator(generator=training_generator,
#                     validation_data=validation_generator,
#                    epochs=2, 
#                    nb_worker=4)
# print("Test-Accuracy:", np.mean(results.history["val_acc"]))


# model.fit_generator(
#         generator=training_generator,
#         samples_per_epoch=2,
#         steps_per_epoch=2000,
#         epochs=50,
#         validation_data=validation_generator,
#         nb_worker=1,
#         validation_steps=800)

/home/ubuntu/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:38: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/home/ubuntu/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:38: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(validation_data=<__main__...., workers=4, callbacks=[<keras.ca..., epochs=2, generator=<__main__....)`


Epoch 1/2
97/97 [==============================] - 1255s 13s/step - loss: 31732.2560 - activation_block_loss: -0.0225 - survival_block_loss: 158661.3910 - activation_block_acc: 0.0819 - activation_block_weighted_dice_coefficient: 0.0225 - activation_block_dice_coefficient: 0.0226 - survival_block_acc: 0.0000e+00 - val_loss: 20088.5926 - val_activation_block_loss: -0.0257 - val_survival_block_loss: 100443.0898 - val_activation_block_acc: 0.0057 - val_activation_block_weighted_dice_coefficient: 0.0257 - val_activation_block_dice_coefficient: 0.0260 - val_survival_block_acc: 0.0000e+00
Epoch 2/2
97/97 [==============================] - 1243s 13s/step - loss: 27350.3229 - activation_block_loss: -0.0242 - survival_block_loss: 136751.7323 - activation_block_acc: 0.0398 - activation_block_weighted_dice_coefficient: 0.0242 - activation_block_dice_coefficient: 0.0245 - survival_block_acc: 0.0000e+00 - val_loss: 24258.8680 - val_activation_block_loss: -0.0270 - val_survival_block_loss: 121294.47

KeyError: 'val_acc'

In [45]:
results.history

{'activation_block_acc': [0.08192945471436706, 0.039846434757790344],
 'activation_block_dice_coefficient': [0.022646129104913662,
  0.024462356325719005],
 'activation_block_loss': [-0.022523154636614563, -0.024239159357186752],
 'activation_block_weighted_dice_coefficient': [0.022523154636614563,
  0.024239159357186752],
 'loss': [31732.25596557696, 27350.322899690607],
 'survival_block_acc': [0.0, 0.0],
 'survival_block_loss': [158661.39098782884, 136751.73229009096],
 'val_activation_block_acc': [0.005741825798482057, 0.02024765409130071],
 'val_activation_block_dice_coefficient': [0.02598769621302684,
  0.0272890323041273],
 'val_activation_block_loss': [-0.025715374110621355, -0.027022822981789],
 'val_activation_block_weighted_dice_coefficient': [0.025715374110621355,
  0.027022822981789],
 'val_loss': [20088.59258544303, 24258.86802391779],
 'val_survival_block_acc': [0.0, 0.0],
 'val_survival_block_loss': [100443.08982942786, 121294.47131166005]}

In [46]:
# score = model.evaluate(x_test, y_test, verbose=0)

params = {'dim': (160,192,160),
          'batch_size': 1,
          'n_classes': 3,
          'n_channels': 4,
          'shuffle': False}

# Turned shuffle off so that we can match the values in the dictionary to the predictions. 
# This way we can compare the predictions side-by-side with the ground truth.

validation_generator = DataGenerator(partition['holdout'], **params)

prediction = model.predict_generator(generator=validation_generator)
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])
prediction

[array([[[[[0.4107392 , 0.4109242 , 0.41286734, ..., 0.414239  ,
            0.4062945 , 0.4087559 ],
           [0.4076335 , 0.4029764 , 0.40531975, ..., 0.40753105,
            0.40047157, 0.40665305],
           [0.40970016, 0.4054101 , 0.4089932 , ..., 0.40780285,
            0.40005317, 0.40587232],
           ...,
           [0.41120982, 0.4073081 , 0.40919992, ..., 0.4117663 ,
            0.40317905, 0.4061273 ],
           [0.40349078, 0.39844176, 0.39888728, ..., 0.40212226,
            0.40201744, 0.40717614],
           [0.40823942, 0.40427506, 0.40381655, ..., 0.405494  ,
            0.40509576, 0.41154116]],
 
          [[0.40704864, 0.40752777, 0.41046774, ..., 0.41235796,
            0.4046494 , 0.40707505],
           [0.39524266, 0.3930453 , 0.39696887, ..., 0.4007749 ,
            0.39224547, 0.4005762 ],
           [0.39786884, 0.39640248, 0.399847  , ..., 0.40040424,
            0.39259943, 0.3974053 ],
           ...,
           [0.3995429 , 0.3972479 , 0.3993155 ,

In [53]:
# prediction[0] are the segmentation masks
# prediction[1] are the days of survival
prediction[1][1]

array([185.06418], dtype=float32)

In [54]:
import pickle

pickle.dump( prediction, open( "prediction2.pkl", "wb" ) )

In [17]:
# # sanity check on the predictions:
# len(prediction)
# prediction[0].shape
# np.unique(prediction)

(3, 160, 192, 160)

In [25]:
# ! mkdir to_categorical_try
# ! mkdir channel_split

mkdir: cannot create directory ‘channel_split’: File exists


In [26]:
# import pickle

# pickle.dump( partition, open( "./channel_split/partition.pkl", "wb" ) ) # this has the test/train ID matches

# # # access the test list:
# # testIDlist = partition['test']
# # testIDlist

In [33]:
# for i in range(len(prediction)):
#     pickle.dump( prediction[i], open( "./channel_split/prediction_"+str(i)+".pkl", "wb" ) )

In [28]:
# # https://keras.io/getting-started/faq/#how-can-i-save-a-keras-model
# # pickle.dump( model, open( "model.pkl", "wb" ) )
# model.save_weights('./channel_split/my_model_weights.h5')

In [2]:
import pickle

partition = pickle.load(open( "./channel_split/partition.pkl", "rb" ) ) # this has the test/train ID matches

# access the test list:
testIDlist = partition['test']
# testIDlist

### Write images to .tif:

In [ ]:
from tifffile import imsave
for i in range(len(prediction)):
    imarray = pickle.load(open( "./channel_split/prediction_"+str(i)+".pkl", "rb" ) )

    imarray[]
    imarray *= 255.0/imarray.max()
    print(np.unique())
    
    imsave("./channel_split/"+testIDlist[i]+"prediction.tif", imarray)

    # make ground truth 
    ID = testIDlist[i]
    img1 = './data/' + ID + '_flair.nii.gz'
    img2 = './data/' + ID + '_t1.nii.gz'
    img3 = './data/' + ID + '_t1ce.nii.gz'
    img4 = './data/' + ID + '_t2.nii.gz'
    img5 = './data/' + ID + '_seg.nii.gz'

    newimage = nib.concat_images([img1, img2, img3, img4, img5])
    cropped = crop_img(newimage)         
    img_array = np.array(cropped.dataobj)
    z = np.rollaxis(img_array, 3, 0)

    padded_image = np.zeros((5,160,192,160))
    padded_image[:z.shape[0],:z.shape[1],:z.shape[2],:z.shape[3]] = z

    a,b,c,d,seg_mask = np.split(padded_image, 5, axis=0)

    images = np.concatenate([a,b,c,d], axis=0)
    imsave("./channel_split/"+testIDlist[i]+"ground_truth.tif", images)
    

### Testing:

In [25]:
import pickle
import numpy as np
import copy
import nibabel as nib

i = 0

imarray = pickle.load(open( "./channel_split/prediction_"+str(i)+".pkl", "rb" ) )
# threshold the channels (for prediction):
prediction_thresh = copy.deepcopy(imarray)
prediction_thresh[prediction_thresh < 0.5] = 0.
prediction_thresh[prediction_thresh >= 0.5] = 1.
prediction_thresh = prediction_thresh
print(np.unique(prediction_thresh))
prediction_thresh *= 255.0/prediction_thresh.max() # convert to 8-bit pixel values
prediction_thresh = prediction_thresh.astype(int)
print(np.unique(prediction_thresh))
print(prediction_thresh.shape)

ID = testIDlist[i]
img1 = './data/' + ID + '_flair.nii.gz'
flairimg = nib.load(img1)
flairimg = np.array(flairimg.dataobj)
flairimg = np.expand_dims(flairimg, axis=0)
flairimg = np.rollaxis(flairimg, 3, 0)
print(np.unique(flairimg))
flairimg = flairimg.astype(float)
flairimg *= 255.0/flairimg.max() # convert to 8-bit pixel values
flairimg = flairimg.astype(int)
print(np.unique(flairimg))
print(flairimg.shape)

[0. 1.]
[  0 255]
(3, 160, 192, 160)
[   0    1    2    3    4    5    6    7    8    9   10   11   12   13
   14   15   16   17   18   19   20   21   22   23   24   25   26   27
   28   29   30   31   32   33   34   35   36   37   38   39   40   41
   42   43   44   45   46   47   48   49   50   51   52   53   54   55
   56   57   58   59   60   61   62   63   64   65   66   67   68   69
   70   71   72   73   74   75   76   77   78   79   80   81   82   83
   84   85   86   87   88   89   90   91   92   93   94   95   96   97
   98   99  100  101  102  103  104  105  106  107  108  109  110  111
  112  113  114  115  116  117  118  119  120  121  122  123  124  125
  126  127  128  129  130  131  132  133  134  135  136  137  138  139
  140  141  142  143  144  145  146  147  148  149  150  151  152  153
  154  155  156  157  158  159  160  161  162  163  164  165  166  167
  168  169  170  171  172  173  174  175  176  177  178  179  180  181
  182  183  184  185  186  187  188  189

### Making ground truth .tiff files: 
- Testing:

In [8]:
import numpy as np
import copy
import nibabel as nib

from tifffile import imsave
from libtiff import TIFF

from skimage.io._plugins import freeimage_plugin as fi

# import javabridge
# import bioformats
# javabridge.start_vm(class_path=bioformats.JARS)

# your program goes here


# ID = testIDlist[i]
# for i in range(len(testIDlist)):
for i in range(2):

    print("current image:", i)

    ID = testIDlist[i]
    img1 = './data/' + ID + '_flair.nii.gz'
    img2 = './data/' + ID + '_t1.nii.gz'
    img3 = './data/' + ID + '_t1ce.nii.gz'
    img4 = './data/' + ID + '_t2.nii.gz'
    img5 = './data/' + ID + '_seg.nii.gz'

    newimage = nib.concat_images([img1, img2, img3, img4, img5])
    cropped = crop_img(newimage)
    img_array = np.array(cropped.dataobj)
    z = np.rollaxis(img_array, 3, 0)

    padded_image = np.zeros((5, 160, 192, 160))
    padded_image[:z.shape[0], :z.shape[1], :z.shape[2], :z.shape[3]] = z

    a, b, c, d, seg_mask = np.split(padded_image, 5, axis=0)

    images = np.concatenate([a, b, c, d], axis=0)

    # print("images shape:", images.shape, "images values:", np.unique(images.astype(int)))

    # split the channels:
    # seg_mask_1 = copy.deepcopy(seg_mask.astype(int))
    seg_mask_1 = np.zeros((1, 160, 192, 160))
    seg_mask_1[seg_mask.astype(int) > 0] = 1
    seg_mask_2 = np.zeros((1, 160, 192, 160))
    seg_mask_2[seg_mask.astype(int) > 1] = 1
    seg_mask_3 = np.zeros((1, 160, 192, 160))
    seg_mask_3[seg_mask.astype(int) > 2] = 1
    seg_mask_3ch = np.concatenate(
        [seg_mask_1, seg_mask_2, seg_mask_3], axis=0).astype(int)

    # def scale_image(image_array):
    #     image_array = image_array.astype(float)
    #     image_array *= 255.0/image_array.max() # convert to 8-bit pixel values
    #     image_array = image_array.astype(int)
    #     return image_array

    # img_array_list = [a,seg_mask_1,seg_mask_2,seg_mask_3]
    # for img_array in img_array_list:
    #     img_array = scale_image(img_array)

    a = a.astype(float)
    a *= 255.0/a.max()  # convert to 8-bit pixel values
    a = np.rollaxis(a, 0, 2)
    a = a.astype('uint8')
#     print("unique flair values:", np.unique(a))

    seg_mask_1 = seg_mask_1.astype(float)
    seg_mask_1 *= 255.0/seg_mask_1.max()  # convert to 8-bit pixel values
    seg_mask_1 = np.rollaxis(seg_mask_1, 0, 2)
    seg_mask_1 = seg_mask_1.astype('uint8')
#     print("unique segment mask values:", np.unique(seg_mask_1))

    seg_mask_2 = seg_mask_2.astype(float)
    seg_mask_2 *= 255.0/seg_mask_2.max()  # convert to 8-bit pixel values
    seg_mask_2 = np.rollaxis(seg_mask_2, 0, 2)
    seg_mask_2 = seg_mask_2.astype('uint8')

    seg_mask_3 = seg_mask_3.astype(float)
    seg_mask_3 *= 255.0/seg_mask_3.max()  # convert to 8-bit pixel values
    seg_mask_3 = np.rollaxis(seg_mask_3, 0, 2)
    seg_mask_3 = seg_mask_3.astype('uint8')

#     ground_truth = np.concatenate(
#         [a, seg_mask_1, seg_mask_2, seg_mask_3], axis=0).astype('uint8')

#     print("unique flair + segment mask values:", np.unique(ground_truth))
    # shape.ground_truth
    # flairimg = flairimg.astype(float)
    # flairimg *= 255.0/flairimg.max() # convert to 8-bit pixel values
    # flairimg = flairimg.astype(int)
    # print(np.unique(flairimg))
#     print("final image shape:", ground_truth.shape)
#     imsave("./channel_split/"+testIDlist[i]+"ground_truth.tif", ground_truth, 'imagej')
    imsave("./channel_split/"+testIDlist[i]+"flair.tif", a, 'imagej')
    imsave("./channel_split/"+testIDlist[i]+"ground_truth_1.tif", seg_mask_1, 'imagej')
    imsave("./channel_split/"+testIDlist[i]+"ground_truth_2.tif", seg_mask_2, 'imagej')
    imsave("./channel_split/"+testIDlist[i]+"ground_truth_3.tif", seg_mask_3, 'imagej')

#     tiff = TIFF.open("./channel_split/"+testIDlist[i]+"ground_truth.tif", mode='w')
#     tiff.write_image(ground_truth)
#     tiff.close()
#     fi.write_multipage(ground_truth, "./channel_split/"+testIDlist[i]+"ground_truth.tif")
#     bioformats.write_image(pathname="./channel_split/"+testIDlist[i]+"ground_truth.tif", 
#                            pixels=ground_truth, 
#                            pixel_type=u'uint8',
#                            size_c=4, size_z=160, size_t=1,
#                            channel_names=None)

# javabridge.kill_vm()

current image: 0
current image: 1


### Adding predictions:

In [24]:
import numpy as np
import copy
import nibabel as nib

from tifffile import imsave
from libtiff import TIFF

from skimage.io._plugins import freeimage_plugin as fi

# import javabridge
# import bioformats
# javabridge.start_vm(class_path=bioformats.JARS)

# your program goes here


# ID = testIDlist[i]
# for i in range(len(testIDlist)):
for i in range(2):

    print("current image:", i)

    ID = testIDlist[i]
    img1 = './data/' + ID + '_flair.nii.gz'
    img2 = './data/' + ID + '_t1.nii.gz'
    img3 = './data/' + ID + '_t1ce.nii.gz'
    img4 = './data/' + ID + '_t2.nii.gz'
    img5 = './data/' + ID + '_seg.nii.gz'

    newimage = nib.concat_images([img1, img2, img3, img4, img5])
    cropped = crop_img(newimage)
    img_array = np.array(cropped.dataobj)
    z = np.rollaxis(img_array, 3, 0)

    padded_image = np.zeros((5, 160, 192, 160))
    padded_image[:z.shape[0], :z.shape[1], :z.shape[2], :z.shape[3]] = z

    a, b, c, d, seg_mask = np.split(padded_image, 5, axis=0)

    images = np.concatenate([a, b, c, d], axis=0)

    # print("images shape:", images.shape, "images values:", np.unique(images.astype(int)))

    # split the channels:
    # seg_mask_1 = copy.deepcopy(seg_mask.astype(int))
    seg_mask_1 = np.zeros((1, 160, 192, 160))
    seg_mask_1[seg_mask.astype(int) > 0] = 1
    seg_mask_2 = np.zeros((1, 160, 192, 160))
    seg_mask_2[seg_mask.astype(int) > 1] = 1
    seg_mask_3 = np.zeros((1, 160, 192, 160))
    seg_mask_3[seg_mask.astype(int) > 2] = 1
    seg_mask_3ch = np.concatenate(
        [seg_mask_1, seg_mask_2, seg_mask_3], axis=0).astype(int)

    # def scale_image(image_array):
    #     image_array = image_array.astype(float)
    #     image_array *= 255.0/image_array.max() # convert to 8-bit pixel values
    #     image_array = image_array.astype(int)
    #     return image_array

    # img_array_list = [a,seg_mask_1,seg_mask_2,seg_mask_3]
    # for img_array in img_array_list:
    #     img_array = scale_image(img_array)

    a = a.astype(float)
    a *= 255.0/a.max()  # convert to 8-bit pixel values
    a = np.rollaxis(a, 0, 2) # cxyz -> xycz for imagej
    a = np.rollaxis(a, 0, 3) # switching x and z
    a = a.astype('uint8')
#     print("unique flair values:", np.unique(a))

    seg_mask_1 = seg_mask_1.astype(float)
    seg_mask_1 *= 255.0/seg_mask_1.max()  # convert to 8-bit pixel values
    seg_mask_1 = np.rollaxis(seg_mask_1, 0, 2) # cxyz -> xycz for imagej
    seg_mask_1 = np.rollaxis(seg_mask_1, 0, 3) # switching x and z
    seg_mask_1 = seg_mask_1.astype('uint8')
#     print("unique segment mask values:", np.unique(seg_mask_1))

    seg_mask_2 = seg_mask_2.astype(float)
    seg_mask_2 *= 255.0/seg_mask_2.max()  # convert to 8-bit pixel values
    seg_mask_2 = np.rollaxis(seg_mask_2, 0, 2) # cxyz -> xycz for imagej
    seg_mask_2 = np.rollaxis(seg_mask_2, 0, 3) # switching x and z
    seg_mask_2 = seg_mask_2.astype('uint8')

    seg_mask_3 = seg_mask_3.astype(float)
    seg_mask_3 *= 255.0/seg_mask_3.max()  # convert to 8-bit pixel values
    seg_mask_3 = np.rollaxis(seg_mask_3, 0, 2) # cxyz -> xycz for imagej
    seg_mask_3 = np.rollaxis(seg_mask_3, 0, 3) # switching x and z
    seg_mask_3 = seg_mask_3.astype('uint8')

#     ground_truth = np.concatenate(
#         [a, seg_mask_1, seg_mask_2, seg_mask_3], axis=0).astype('uint8')

#     print("unique flair + segment mask values:", np.unique(ground_truth))
    # shape.ground_truth
    # flairimg = flairimg.astype(float)
    # flairimg *= 255.0/flairimg.max() # convert to 8-bit pixel values
    # flairimg = flairimg.astype(int)
    # print(np.unique(flairimg))
#     print("final image shape:", ground_truth.shape)
#     imsave("./channel_split/"+testIDlist[i]+"ground_truth.tif", ground_truth, 'imagej')
    imsave("./channel_split/"+testIDlist[i]+"_flair.tif", a, 'imagej')
    imsave("./channel_split/"+testIDlist[i]+"_ground_truth_1.tif", seg_mask_1, 'imagej')
    imsave("./channel_split/"+testIDlist[i]+"_ground_truth_2.tif", seg_mask_2, 'imagej')
    imsave("./channel_split/"+testIDlist[i]+"_ground_truth_3.tif", seg_mask_3, 'imagej')

    imarray = pickle.load(open( "./channel_split/prediction_"+str(i)+".pkl", "rb" ) )

    prediction_thresh = copy.deepcopy(imarray)
    prediction_thresh[prediction_thresh < 0.5] = 0.
    prediction_thresh[prediction_thresh >= 0.5] = 1.
    prediction_thresh = prediction_thresh
    print(np.unique(prediction_thresh))
    prediction_thresh *= 255.0/prediction_thresh.max() # convert to 8-bit pixel values
    prediction_thresh = prediction_thresh.astype('uint8')
    prediction_thresh = np.rollaxis(prediction_thresh, 1, 3) # switching x and z; c will be taken care of in split
    print(np.unique(prediction_thresh))
    print(prediction_thresh.shape)

    pred1, pred2, pred3 = np.split(prediction_thresh, 3, axis=0)

    imsave("./channel_split/"+testIDlist[i]+"_predicted_1.tif", pred1, 'imagej')
    imsave("./channel_split/"+testIDlist[i]+"_predicted_2.tif", pred2, 'imagej')
    imsave("./channel_split/"+testIDlist[i]+"_predicted_3.tif", pred3, 'imagej')

    # print("images shape:", images.shape, "images values:", np.unique(images.astype(int)))

    # split the channels:
    # seg_mask_1 = copy.deepcopy(seg_mask.astype(int))


    #     seg_mask_3ch = np.concatenate(
#         [seg_mask_1, seg_mask_2, seg_mask_3], axis=0).astype(int)

#     imarray *= 255.0/imarray.max()

#     imsave("./channel_split/"+testIDlist[i]+"ground_truth_1.tif", seg_mask_1, 'imagej')
#     imsave("./channel_split/"+testIDlist[i]+"ground_truth_2.tif", seg_mask_2, 'imagej')
#     imsave("./channel_split/"+testIDlist[i]+"ground_truth_3.tif", seg_mask_3, 'imagej')

current image: 0
[0. 1.]
[  0 255]
(3, 192, 160, 160)
current image: 1
[0. 1.]
[  0 255]
(3, 192, 160, 160)
